In [2]:
import random
import numpy as np
from evaluate import load

# Load metrics
meteor = load("meteor")
# ter = load("ter")
comet = load("comet")
# bertscore = load("bertscore")
# bluert = load("bleurt")
fluency = load("Abdulmohsena/classicier")
wer = load("wer")
# DPO for human alignment

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\user\.cache\huggingface\hub\models--Unbabel--wmt22-comet-da\snapshots\371e9839ca4e213dde891b066cf3080f75ec7e72\checkpoints\model.ckpt`


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

d:\anaconda\envs\Faseeh\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--xlm-roberta-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Encoder model frozen.
d:\anaconda\envs\Faseeh\Lib\site-packages\pytorch_lightning\core\saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [5]:
fluency.compute(texts=['هلا', 'وأما المدان وغيره من أطراف الدعوى فيعطون نسخة مصدَّقة عن ذلك القرار، ويجوز أن تحذف منه توقيعات القضاة الذين شاركوا في القرار.'])

{'classical_score': array([0.95342326, 0.00630969], dtype=float32)}

In [3]:
def all_metrics(predictions, references, sources):
    import warnings
    warnings.filterwarnings('ignore')
    results = {}
    results['meteor'] = meteor.compute(predictions=predictions, references=references)
    results['comet'] = comet.compute(predictions=predictions, references=references, sources=None)
    results['is_classical'] = fluency.compute(texts=predictions)
    results['wer'] = wer.compute(predictions=predictions, references=references)
    
    return results

In [4]:
predictions = ['وها هنا نقول أن العضة من الكلب', 'مرحبا', 'قالوا انا كنا معرضين', "ما طار طير وارتفع الا كما طار وقع"]
references = ['وقلنا أن العض من الكلب', 'أهلا', "أعرضنا عن هذا", "ما طار طير وارتفع، الا كما طار وقع"]
sources = ['and we said the bite is from the dog', 'hi', 'we were ignorant', "hi"]

In [19]:
import numpy as np


In [6]:
predictions = ['المشروع وغير المشروع واضحان']
references = [dediac.dediac_ar('الحلال بين والحرام بين')]
sources = ['Lawful and unlawful are clear']

NameError: name 'dediac' is not defined

In [5]:
all_metrics(predictions, references, sources)

TypeError: object of type 'NoneType' has no len()

In [10]:
comet.compute(predictions=predictions, references=references, sources=sources)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'mean_score': 0.7756509780883789,
 'scores': [0.8530154824256897,
  0.8073979020118713,
  0.5271443724632263,
  0.9150461554527283]}

In [29]:
from datasets import load_dataset



from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
model_name = "AbdulmohsenA/Faseeh"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang="eng_Latn", tgt_lang="arb_Arab")

dataset = load_dataset("Abdulmohsena/Classic-Arabic-English-Language-Pairs")['train']


README.md:   0%|          | 0.00/869 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.59M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26486 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5298 [00:00<?, ? examples/s]

In [43]:
preprocess_function = lambda examples: tokenizer(
    examples['en'], text_target=examples['ar'], max_length=256, truncation=True, padding=True, return_tensors='pt')
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['en', 'ar']).shuffle()
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.20)
import torch
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True, return_tensors='pt')
eval_dataloader = torch.utils.data.DataLoader(tokenized_dataset["test"], batch_size=4, collate_fn=data_collator, shuffle=True)

Map:   0%|          | 0/26486 [00:00<?, ? examples/s]

In [47]:
sample = next(iter(eval_dataloader))

In [48]:
tokenizer.batch_decode(sample['input_ids'], skip_special_tokens=True)

['When this went on for a long time, the man came to Abu al-Fath, and there was a piece of bread under the bowl, and said: O Abu al-Fath, take this loaf, cut it up, and divide it among our companions. Abu al-Fath ignored him, then he repeated the words to him, and he ignored him, and when he repeated the words to him, he ignored him, and when he repeated the words to him a fourth time, he said: What is the matter with you, woe to you, that you do not cut it up among them?',
 'Nothing is heavier upon the scale of the believer on the Day of Resurrection than his good character. Verily, Allah hates the shameless, obscene person.',
 'Then they agreed and left, and one of them refused to help them and to enter into the debt with them.',
 'Patience is not silence and stillness, nor is it a lack of shouting and grumbling.']

In [49]:
tokenizer.batch_decode(sample['labels'], skip_special_tokens=True)

['فلما طال ذلك عليهم أقبل الرجل على أبي الفتح، وتحت القصعة رقاقة ، فقال: يا أبا الفتح، خذ ذلك الرغيف فقطعه، واقسمه على أصحابنا فتغافل أبو الفتح، ثم أعاد عليه القول: فتغافل، فلما أعاد عيه القول: فتغافل، فلما أعاد عليه القول، الرابعة، قال: مالك، ويلك، لا تقطعه بينهم؟',
 'ما شيء أثقل في ميزان المؤمن يوم القيامة من خلق حسن وإن الله ليبغض الفاحش البذيء',
 'ثم أنهم تناهدوا وتخارجوا ، وأبى واحد منهم أن يعينهم، وأن يدخل في الغرم معهم.',
 'وليس الصبر بالصمت والسكوت، ولا بقلة الصياح والضموز.']

In [51]:
output = model(**sample)

In [55]:
tokenizer.batch_decode(output.logits.argmax(dim=-1), skip_special_tokens=True)

['فلما طال ذلك،،بل الرجل على أبي الفتح، وإحت القصعة رقاقة، فقال: يايا أبا الفتح، خذ هذا الرغيف،عه، واقسمه على أصحابنا، ،غافله أبو الفتح، ثم أعاده عليه الكول، فتغافل، فلما أعاد عليهنده القول، فتغافل، فلما أعاد عليه القول، فلابعة قال قال: ماك يا ويلك، لا تقطعه بينهم؟',
 'ما شيء أثقل في ميزان المؤمن يوم القيامة من حسن حسن وإن الله ليبغض الفاحش الفذيء',
 'ثم توا واهدوا وانولوا، وأبى أح منهم المس يعينهم، وأن يدخل مع الدر معهم.',
 'وليس الصبر بالسكمت والسكينة، ولا بقلة الصياح والحدير.']